# Attractors and reachability analysis @ ECCB2020-T05
<div style="text-align: right; font-style: italic;">Pedro T. Monteiro</div>

You can obtain information about the:
- CoLoMoTo consortium at [http://colomoto.org](http://colomoto.org)
- CoLoMoTo notebook at [doi:10.3389/fphys.2018.00680](https://doi.org/10.3389/fphys.2018.00680)

***

Here, we consider the following list of software tools:
- **bioLQM**: library which provides a datastructure for the representation of logical models, and a collection of filters to load or save these models in various formats, facilitating the interoperability between tools which do not directly support SBML qual. It relies on JSBML for SBML qual support. It can be used on the command line for model conversion or stable state identification. It can also provide its "core" to other tools (such as GINsim, CoLoMoTo-docker and Epilog) which can benefit from additional API for the representation of perturbations or simulation engines.
  - to compute the stable states
- **GINsim**: Java application for the construction and analysis of multi-valued logical models, which provides a user-friendly graphical interface to define the model itself.
  - to display the regulatory graph
- **boolsim**: BoolSim implements an efficient algorithm, based on reduced ordered binary decision diagrams to evaluate attractors (stable states, as well as cyclic attractors) and perform perturbation experiments on Boolean networks, using synchronous or asynchronous network dynamics.
  - to compute both stable states and complex attractors

Most examples below were borrowed from [https://github.com/colomoto/colomoto-docker](https://github.com/colomoto/colomoto-docker), which is mainly maintained by Loïc Paulevè and Aurélien Naldi.

***
### Attractor identification

You can import all the necessary tools/libraries at the beginning or later on as needed.

In [33]:
import biolqm

The [GINsim.org](http://ginsim.org) website provides not only the latest GINsim software, but also a [Model repository](http://ginsim.org/models_repository) with numerous models that can be browsed and search by name, taxon or process.

In [34]:
lqm = biolqm.load("http://ginsim.org/sites/default/files/phageLambda4.zginml")

Downloading http://ginsim.org/sites/default/files/phageLambda4.zginml

Optionaly, you can visualize the regulatory graph of the model. However, bioLQM does not contain visualization features and we need to use (a headless version of) GINsim to output the image of the graph.

In [35]:
import ginsim
lrg = biolqm.to_ginsim(lqm)
ginsim.show(lrg)

In [48]:
bioLQM_fixpoints = biolqm.fixpoints(lqm)
print('# fixpoints:', len(bioLQM_fixpoints))

from colomoto_jupyter import tabulate
tabulate(bioLQM_fixpoints)

# fixpoints: 1


,CI,Cro,CII,N
0,2,0,0,0


But wait! Phage Lambda model was supposed to have two attractors, right? One denoting lysis, and another denoting lysogeny. So, is the model not generating it or is it just "disguised" as a complex attractor?

There is no efficient method to identify complex attractors. This means that we have to perform simulations, i.e., have to explicitly generate the dynamics.
Although bioLQM/GINsim efficiently discover stable states (without simulation), they do not scale for the explicit generation of huge state transition graphs (STGs).

Here, we consider boolsim, which albeit doing a simulation based identification of attractors, it does it very efficiently (more info at [doi:10.1093/bioinformatics/btn336](https://doi.org/10.1093/bioinformatics/btn336).

In [124]:
import boolsim
boolsim_attractors = boolsim.attractors(lqm)
print('# attractors:', len(boolsim_attractors))
tabulate(boolsim_attractors)

# attractors: 2


,CI_b1,CI_b2,CII,Cro_b1,Cro_b2,Cro_b3,N
0,0,0,0,1,1,*,0
1,1,1,0,0,0,0,0


Indeed, we recover the previous stable state with high level of CI (lysogenic state), plus a complex attractor with oscillating high levels of Cro (lysis).

***
### Attractor reachability

Even if a model is capable of generating the correct attractors, it still needs to ensure that each one is reached from specific initial conditions, characterized by a (set of) state(s).

In [52]:
init = {str(node): 0 for node in lrg.getNodes()}
init_reached = boolsim.reachable(lqm, init)
tabulate(init_reached)

,CI_b1,CI_b2,CII,Cro_b1,Cro_b2,Cro_b3,N
0,0,*,0,0,*,*,*
2,0,*,0,1,*,*,*
1,0,*,1,0,*,*,1
3,0,*,1,1,0,*,1
4,0,*,1,1,1,*,*
5,1,*,*,*,*,*,*


So, did we reach any of the attractors? This is one of the smallest models in the model repository. What if we were working with a model with 100 nodes?

Also, note that "reached states" are all the states visited leading to attractors, not only the states belonging to the attractors.

We need to verify if any of the attractors intersects with the set of reached states. This is done using:
 - **boolsim_setutils**: a tool to efficiently perform the union, difference and intersection between sets of states.

In [66]:
# The initial state is contained in the set of reached states
print(boolsim.intersection(init, init_reached))

{'CI': 0, 'Cro': 0, 'CII': 0, 'N': 0, 'CI_b1': '*', 'CI_b2': '*', 'Cro_b1': '*', 'Cro_b2': '*', 'Cro_b3': '*'}


In [67]:
# The lytic states are contained in the set of reached states
print(boolsim.intersection(boolsim_attractors[0], init_reached))
# The lysogenic state is contained in the set of reached states
print(boolsim.intersection(boolsim_attractors[1], init_reached))

{'CI_b1': 0, 'CI_b2': 0, 'CII': 0, 'Cro_b1': 1, 'Cro_b2': 1, 'Cro_b3': '*', 'N': 0}
{'CI_b1': 1, 'CI_b2': 1, 'CII': 0, 'Cro_b1': 0, 'Cro_b2': 0, 'Cro_b3': 0, 'N': 0}


In [72]:
# If we start from the Lytic attractor states, we (obviously) cannot reach the lysogenic state
lysis_reached = boolsim.reachable(lqm, boolsim_attractors[0])
print(boolsim.intersection(boolsim_attractors[1], lysis_reached))

None


***
### Property verification with NuSMV

Sometimes we need to verify not only that an attractor is present and it is reachable, but also that from the initial state until the attractor an additional condition is met. For example, it passes through an additional state.

To verify conditions on trajectories, we use:
- **NuSMV**: a BDD-based symbolic model-checker [http://nusmv.fbk.eu](http://nusmv.fbk.eu).

The expression of conditions on trajectories is performed through the use of temporal properties, in particular, using Computational Tree Logic (CTL) or Linear Temporal Logic (LTL) properties.

We use the `colomoto` python module which offers a generic interface for declaring temporal properties using either LTL or CTL.

In [95]:
from colomoto.temporal_logics import *

#### Defining properties

Properties on states are specified using the `S` operator.
The following property characterize the initial state when all the nodes are inactive:

In [96]:
initial_state = S(CI=0,CII=0,Cro=0,N=0)

The lysogenic state is characterized by the node `CI` being permanently strongly expressed, which is modeled by `CI=2`. The following CTL property expresses that from the given state, all the reachable states (`AG`) satisfy `CI=2`:

In [97]:
lysogenic = AG(S(CI=2))

The lytic state is characterized by the node `CI` being inactive (value 0), and `Cro` oscillating between activity levels 2 and 3. This is stated by the following CTL property:

In [98]:
lytic = AG(S(CI=0) & (S(Cro=2) | S(Cro=3)))

The following CTL properties is true if all the reachable attractors are either lysogenic or lytic states:

In [99]:
attractors = AG(EF(lysogenic | lytic))

#### Calling NuSMV

GINsim provides a translation of Boolean and multivalued networks into NuSMV models:

In [100]:
smv = ginsim.to_nusmv(lrg)

Here, `smv` is a Python object representing the NuSMV model.
CTL/LTL specification can be added using the method `add_ctl` and `add_ltl`, respectively.
For convenience, a label can be given to a property:

In [101]:
smv.add_ctl(attractors, name="global_attractors")

Properties can also be added by bulk, providing a Python dictionnary specifying the properties.
In the following cell, we specify two properties: one verifying that, from the initial state, there exists a trajectory leading to a stable lysogenic state, and another one verifying that, from the initial state, there exists a trajectory leading to a stable lytic state.

In [102]:
specs = {
    "reach_lyso": If(initial_state, EF(lysogenic)),
    "reach_lytic": If(initial_state, EF(lytic))
}
smv.add_ctls(specs)

The method `verify` will execute NuSMV and returns the verification result for each property:

In [103]:
smv.verify()

{'global_attractors': True, 'reach_lyso': True, 'reach_lytic': True}

We could also verify if starting from the initial state, it is possible to pass through some intermediate state before reaching both attractors.

In [122]:
middle_CI = S(CI=1,CII=0,Cro=0,N=0)
middle_Cro = S(CI=0,CII=0,Cro=1,N=0)
specs2 = {
    "middle_CI_lyso": If(initial_state, EF(middle_CI & EF(lysogenic))),
    "middle_Cro_lyso": If(initial_state, EF(middle_Cro & EF(lysogenic))),
    "middle_CI_lytic": If(initial_state, EF(middle_CI & EF(lytic))),
    "middle_Cro_lytic": If(initial_state, EF(middle_Cro & EF(lytic))),
    "middle_Cro_lytic_AF": If(initial_state, AF(middle_Cro & EF(lytic)))
}
smv = ginsim.to_nusmv(lrg)
smv.add_ctls(specs2)

In [123]:
smv.verify()

{'middle_CI_lyso': True,
 'middle_Cro_lyso': True,
 'middle_CI_lytic': True,
 'middle_Cro_lytic': True,
 'middle_Cro_lytic_AF': False}